In [3]:
import os
import zipfile
import rarfile
import shutil
from pathlib import Path

# 设置路径
source_dir = r"C:\Users\gongh\Downloads\ALL_tsp"
output_dir = r"C:\Users\gongh\Downloads\ALL_tsp_extracted"

# 创建输出文件夹
os.makedirs(output_dir, exist_ok=True)

# 临时解压目录
temp_dir = os.path.join(source_dir, "temp_extract")
os.makedirs(temp_dir, exist_ok=True)

def extract_archive(file_path, extract_to):
    """解压压缩文件"""
    try:
        if file_path.endswith('.zip'):
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(extract_to)
            print(f"已解压 ZIP: {os.path.basename(file_path)}")
        elif file_path.endswith(('.rar', '.tsp')):
            # .tsp文件可能是RAR格式的压缩文件
            try:
                with rarfile.RarFile(file_path, 'r') as rar_ref:
                    rar_ref.extractall(extract_to)
                print(f"已解压 RAR: {os.path.basename(file_path)}")
            except:
                # 如果不是RAR格式，可能就是普通TSP文件
                print(f"跳过 {os.path.basename(file_path)} (不是压缩文件)")
        return True
    except Exception as e:
        print(f"解压失败 {os.path.basename(file_path)}: {str(e)}")
        return False

def find_and_copy_tsp(directory, output_folder):
    """递归查找所有TSP文件并复制到输出文件夹"""
    tsp_count = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.tsp'):
                src_path = os.path.join(root, file)
                # 避免文件名冲突，如果存在同名文件则添加数字后缀
                dst_path = os.path.join(output_folder, file)
                base_name = os.path.splitext(file)[0]
                extension = os.path.splitext(file)[1]
                counter = 1
                while os.path.exists(dst_path):
                    new_name = f"{base_name}_{counter}{extension}"
                    dst_path = os.path.join(output_folder, new_name)
                    counter += 1
                
                shutil.copy2(src_path, dst_path)
                print(f"已复制: {file} -> {os.path.basename(dst_path)}")
                tsp_count += 1
    return tsp_count

# 主流程
print("=" * 60)
print("开始处理压缩文件...")
print("=" * 60)

# 获取所有压缩文件
compressed_files = []
for file in os.listdir(source_dir):
    file_path = os.path.join(source_dir, file)
    if os.path.isfile(file_path) and (file.endswith('.tsp') or file.endswith('.zip') or file.endswith('.rar')):
        compressed_files.append(file_path)

print(f"\n找到 {len(compressed_files)} 个压缩文件\n")

# 解压所有文件
for idx, file_path in enumerate(compressed_files, 1):
    print(f"[{idx}/{len(compressed_files)}] 处理: {os.path.basename(file_path)}")
    
    # 为每个压缩文件创建单独的临时目录
    file_temp_dir = os.path.join(temp_dir, os.path.splitext(os.path.basename(file_path))[0])
    os.makedirs(file_temp_dir, exist_ok=True)
    
    # 解压
    extract_archive(file_path, file_temp_dir)

print("\n" + "=" * 60)
print("开始提取TSP文件...")
print("=" * 60 + "\n")

# 从临时目录中提取所有TSP文件
total_tsp = find_and_copy_tsp(temp_dir, output_dir)

# 清理临时目录
print("\n清理临时文件...")
shutil.rmtree(temp_dir)

print("\n" + "=" * 60)
print(f"完成! 共提取了 {total_tsp} 个TSP文件")
print(f"所有TSP文件保存在: {output_dir}")
print("=" * 60)

开始处理压缩文件...

找到 0 个压缩文件


开始提取TSP文件...


清理临时文件...

完成! 共提取了 0 个TSP文件
所有TSP文件保存在: C:\Users\gongh\Downloads\ALL_tsp_extracted


In [2]:
!pip install rarfile